In [1]:
import sys
sys.path.append('..') # monkey patch paths to be able to see condor package

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.ticker as ticker
from matplotlib import pyplot as plt

from pandas_datareader.nasdaq_trader import get_nasdaq_symbols 
from pandas_datareader import data, wb
import condor.forecast as cd 

# Get a list of all available stock symbols

In [3]:
stock_tickers = get_nasdaq_symbols()
stock_tickers.head()

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.0,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.0,False,NaN,AA,AA,False
AAA,True,Investment Managers Series Trust II AXS First ...,P,,True,100.0,False,NaN,AAA,AAA,False
AAAU,True,Goldman Sachs Physical Gold ETF Shares,Z,,True,100.0,False,NaN,AAAU,AAAU,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.0,False,NaN,AAC,AAC,False


In [4]:
# Look up IBM
stock_tickers.loc[stock_tickers.index == 'IBM']

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
IBM,True,International Business Machines Corporation Co...,N,,False,100.0,False,NaN,IBM,IBM,False


In [5]:
# Look up all stocks on the Nasdqa 'Q' and NYSE 'N'
stock_tickers.loc[stock_tickers['Listing Exchange'].isin(['N','Q'])]

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.0,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.0,False,NaN,AA,AA,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.0,False,NaN,AAC,AAC,False
AAC.U,True,"Ares Acquisition Corporation Units, each consi...",N,,False,100.0,False,NaN,AAC.U,AAC=,False
AAC.W,True,Ares Acquisition Corporation Redeemable Warran...,N,,False,100.0,False,NaN,AAC.WS,AAC+,False
...,...,...,...,...,...,...,...,...,...,...,...
ZXYZ.A,True,Nasdaq Symbology Test Common Stock,Q,Q,False,100.0,True,N,NaN,ZXYZ.A,False
ZXZZT,True,NASDAQ TEST STOCK,Q,G,False,100.0,True,N,NaN,ZXZZT,False
ZYME,True,Zymeworks Inc. - Common Stock,Q,Q,False,100.0,False,N,NaN,ZYME,False


# Read Stock Prices

In [ ]:
s = pd.Series([1, 2, np.nan, 2.5, 6.6, np.nan])
# Replace zeros with NaN values to allow for interpolation 
y_series[y_series == 0] = np.nan

# Interpolate missing values using polynomial interpolation 
x = y_series.index[y_series.notnull()]
y = y_series[y_series.notnull()]
f = interp1d(x, y, kind='cubic', fill_value='extrapolate')
y_series.loc[:] = f(y_series.index).clip(min=0)  #dissalow values to go less than zero
     

In [6]:
s = pd.Series([1, 2, np.nan, 2.5, 6.6, np.nan])
result = cd.smooth_lowess(y_series=s, smoothing_window=4, smoothing_iterations=1)
print(result)

pd.DataFrame(
    {'series':s,
     'result':result
    }
)

ValueError: A value (5.0) in x_new is above the interpolation range's maximum value (4).

In [ ]:
# Start and end times
start = pd.to_datetime('1-1-2022').date()
end = dt.date.today()

# Stock ticker
ticker_symbol = 'IBM'

In [ ]:
# Read stock prices from Google Finance
ibm_prices = data.DataReader(ticker_symbol, 'stooq', start=start, end=end)
ibm_prices.head()

In [ ]:
ibm_prices['close_smooth'] = cd.smooth_lowess(
    y_series=ibm_prices['Close'],
    smoothing_window=15, 
    smoothing_iterations=50
)

# Basic Chart

In [ ]:
dates = ibm_prices.index
closing_prices = ibm_prices['Close']
smoothed_prices = ibm_prices['close_smooth'] 

plt.style.context('ggplot')
fig, ax = plt.subplots(figsize = (15,7), dpi = 200)

fmt = '${x:,.0f}'
tick = ticker.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick) 
ax.tick_params(axis='y', labelsize = 14)
ax.tick_params(axis='x', labelsize = 14)

ax.set_title('\nClosing Prices for {}\n'.format(ticker_symbol), fontsize = 25)
ax.set_xlabel('\nDate\n', fontsize = 20)
ax.set_ylabel('\nPrice\n', fontsize = 20)

ax.scatter(dates, closing_prices, facecolors='none', edgecolors='black', linewidth = 1, s = 35, label = 'Closing Prices')
ax.plot(dates, closing_prices, color = 'dodgerblue', linewidth = 1, alpha = 1, label = 'Actual')
ax.plot(dates, smoothed_prices, color = 'maroon', linewidth = 3, linestyle='--',alpha = 1, label = 'LOWESS Trend')

ax.grid(which = 'major')
plt.legend(loc = 2, fontsize = 18)
plt.show()


In [24]:
from itertools import product
max_p=3
max_d=2
max_q=3
d=None

if d==None: 

    test_params = list(
        product(
            np.arange(0, max_p+1, 1),
            np.arange(0, max_p+1, 1),
            np.arange(0, max_q+1, 1)
                     )
            )

else: 
    test_params = list(
        product(
            np.arange(max_p+1),
            np.array([d]),
            np.arange(max_q+1)
                     )
            )
test_params


IndentationError: expected an indented block after 'if' statement on line 7 (1664324663.py, line 9)

In [18]:
np.zeros(1)

array([0.])

In [17]:
np.arange(max_p+1)

array([0, 1, 2, 3])

In [30]:
def generate_arimax_test_params(
        max_p: int = 3, 
        max_d: int = 2, 
        max_q: int = 3, 
        d: int = None
        ) -> list:
    """
    Generate test parameters for an ARIMAX model.

    Parameters
    ----------
    max_p : int, default=3
        Maximum value for the autoregressive (AR) parameter p.
    max_d : int, default=2
        Maximum value for the differencing parameter d.
    max_q : int, default=3
        Maximum value for the moving average (MA) parameter q.
    d : int or None, default=None
        Value for the differencing parameter d. If None (default), all values from 0 to max_d will be used.

    Returns
    -------
    list of tuple
        List of tuples representing all combinations of p, d and q values to test.

    Examples
    --------
    >>> generate_arimax_test_params()
    [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3), ...]
    
    >>> generate_arimax_test_params(max_p=2)
    [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3), ...]
    """
    
    # Generate ARIMAX test params if d (differencing) is provided
    if d is None:
        test_params = list(product(
            np.arange(max_p + 1),
            np.arange(max_d + 1),
            np.arange(max_q + 1)
        ))
    # Else generate ARIMAX test params if d (differencing) is not provided
    else:
        test_params = list(product(
            np.arange(max_p + 1),
            np.array([d]),
            np.arange(max_q + 1)
        ))

    return test_params


In [32]:
params = generate_arimax_test_params(max_p=2)
print(params)

[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 2, 0), (0, 2, 1), (0, 2, 2), (0, 2, 3), (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 0, 3), (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 2, 0), (1, 2, 1), (1, 2, 2), (1, 2, 3), (2, 0, 0), (2, 0, 1), (2, 0, 2), (2, 0, 3), (2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 2, 0), (2, 2, 1), (2, 2, 2), (2, 2, 3)]


In [33]:
test_params

[(0, 2, 0),
 (0, 2, 1),
 (0, 2, 2),
 (0, 2, 3),
 (1, 2, 0),
 (1, 2, 1),
 (1, 2, 2),
 (1, 2, 3),
 (2, 2, 0),
 (2, 2, 1),
 (2, 2, 2),
 (2, 2, 3),
 (3, 2, 0),
 (3, 2, 1),
 (3, 2, 2),
 (3, 2, 3)]

In [34]:
type(test_params)

list

In [330]:
from statsmodels.tsa.stattools import adfuller
from scipy.optimize import minimize
from scipy.stats import norm
import statsmodels.api as sm

In [374]:
def check_nans_and_zeros(
    y_series: pd.Series
    )->bool:
    """
    Checks if a pandas Series contains any NaN values or zeros.

    Parameters
    ----------
    y_series : pandas.Series
        A pandas Series to check for NaN values or zeros.

    Returns
    -------
    bool
        True if the input Series contains any NaN values or zeros, False otherwise.
    
    Examples
    --------
    >>> s = pd.Series([1, 2, np.nan])
    >>> result = contains_nan_or_zero(s)
    >>> print(result)
    True
    """
    # Create a copy of the input Series to avoid modifying it 
    y_series = y_series.copy()

    # Check for NaN values or zeros in the input Series 
    return y_series.isnull().any() or (y_series == 0).any()

In [332]:
def smooth_lowess(
        y_series: pd.Series,
        smoothing_window: int=15,
        smoothing_iterations: int=2
        )->np.array:
    """
    A custom LOWESS (Locally Weighted Scatterplot Smoothing) implementation that "smooths" discrete stock price data
    to produce a trend. It uses a continuous weighted linear least squares regression in a window as passed over the 
    range of ordered stock values. The stock prices are initially log-transformed to prevent the occurrence of negative
    numbers during smoothing. NaN and zero values are interpolated using polynomial interpolation. Execution stops and
    throws a value error if the window length is greater than the series it's supposed to smooth or less than 3. 
    
    Parameters
    ----------
    y_series: pandas.Series 
        Discrete price points as the inputs variable.
    smoothing_window: int , default = 15
        Window length used to pass through the data set as it's smoothed. It can't be less than three.
    smoothing_iterations: int, default = 2
        The number of residual-based reweightings to perform.
    Returns
    -------
    pandas Series
        A pandas Seris of LOWESS smoothed values.  
    
    Raises
    ------
    ValueError
        If the smoothign window 'smoothing_window' is less than 3 or if it's greater than the length of the time series 'y_series'.

    Examples
    --------
    >>> s = pd.Series([1, 2, np.nan, 2.5, 6.6, np.nan])
    >>> result = cd.smooth_lowess(y_series=s, smoothing_window=4, smoothing_iterations=1)
    >>> print(result)
    """
    # Create a copy of the input Series to avoid modifying it 
    y_series = y_series.copy()

    # Stop execution of the function if the window length is greater than the series or less than 3. 
    if smoothing_window < 3 or smoothing_window > len(y_series):
        print("Error: Smoothing_window is less than 3 or greater than the length of the series.")
        raise ValueError("Invalid value for variable.")

    # Fill NaN values with zeros, inteprolate using linear interpolation, and log transform. 
    if check_nans_and_zeros(y_series):
        print('Warning: Series contains NaNs or Zeros. Interpolating values.')
        y_series = interpolate_stock_prices(y_series) 

    # Execute smoother and return a pandas Series
    y_smooth = np.expm1(                                  # the inverse transform log(x+1)
            np.transpose(                                 # transpose resulting array to separate the values
                sm.nonparametric.lowess(
                    endog=np.log1p(y_series),             # log (x+1) to transform prices
                    exog=np.arange(0,len(y_series),1), 
                    frac=smoothing_window/len(y_series),  # the fraction of the window to the length of the series
                    it=smoothing_iterations
                    )
                )[1]
            )
        
    return pd.Series(y_smooth)

In [333]:
def adf_test(
        y_series: pd.Series,
        critical_val: float=0.5
        )->bool:
    """
    Performs an Augmented Dickey-Fuller (ADF) unit root test for stationarity.

    Parameters
    ----------
    series : pd.Series
        A pandas Series containing the time series data to be tested for stationarity.
    critical_val : float, default=0.5
        Critical value for the test statistic. Usually set at 0.1, 0.5. 10 for the 1%, 5%, and 10% levels 
        respectively.
    Returns
    -------
    bool
        A boolean value indicating whether the time series is stationary or not based on the ADF test result.
    
    Examples
    --------
    >>> dates = pd.date_range('2022-01-01', '2022-01-10')
    >>> data = [100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
    >>> series = pd.Series(data=data,index=dates)
    >>> result = adf_test(series)
    >>> print(result)
    
     """
    
    # Perform the ADF test 
    result = adfuller(y_series)
     
    # Extract the p-value from the test result 
    p_value = result[1]
     
    # Determine whether the time series is stationary based on the p-value 
    return p_value < critical_val

In [334]:
def make_stationary(
    y_series: pd.Series,
    max_diff: int = 2
)-> tuple[pd.Series, int]:
    """
    Makes a time series stationary by differencing.

    Parameters
    ----------
    series : pandas.Series
        A pandas Series containing the time series data to be made stationary.
    
    max_diff : int, optional
        The maximum number of times to difference the time series data (default is 3).

    Returns
    -------
    tuple(pd.Series,int)
        A tuple containing:
            - The resulting differenced time series data if it was made stationary or a copy of original input if it could not be made stationary.
            - The number of times that the time series was differenced.

    Examples
    --------
     >>> dates = pd.date_range('2022-01-01', '2022-01-10')
     >>> data = [100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
     >>> series = pd.Series(data=data,index=dates)
     >>> result = make_stationary(series)
     >>> print(result)
     """
    
    # Create a copy of the input Series to avoid modifying it 
    diff_series = y_series.copy()
     
    # Check if the time series is already stationary 
    is_stationary = adf_test(diff_series)
        
    # Difference the time series data until it is stationary or until max_diff is reached 
    i = 0
    while not is_stationary and i < max_diff:
        diff_series = diff_series.diff().dropna()
        i += 1
        is_stationary = adf_test(diff_series)
     
    # Check if the resulting time series was made stationary 
    if not is_stationary:
        return (y_series, None)
     
    # Return the resulting differenced time series and number of differences 
    return (diff_series, i)

In [369]:
def neg_log_likelihood(
    params: tuple[float, float],
    y_array: np.array
)->float:
    """
    Calculate the negative log-likelihood of the data given the parameters of a normal distribution.

    Parameters
    ----------
    params : array-like
        An array containing the values of the mean and standard deviation parameters of a normal distribution.
    y_array : array-like
        An array of data.

    Returns
    -------
    nll : float
        The value of the negative log-likelihood.

    Examples
    --------
    >>> params = [2.3, 0.88]
    >>> data = [1.2, 2.3, 3.4]
    >>> nll = neg_log_likelihood(params,data)
    >>> print(f"Negative log-likelihood: {nll:.2f}")
    Negative log-likelihood: 1.56
   """
    mu, sigma = params
    nll = -np.sum(np.log(norm.pdf(y_array, mu, sigma)))
    return nll



In [371]:
def estimate_normal_params(
    y_series: pd.Series
)->tuple[float, float]:
    """
    Estimate the parameters of a normal distribution using Maximum Likelihood Estimation (MLE).

    Parameters
    ----------
    y_series : pandas Series
        A pandas Series that may contain missing values (represented by np.nan) or zeros.
    log_transorm: bool, default = True
        True to log transform the series
    Returns
    -------
    mu_estimated : float
        The estimated value of the mean parameter.
    sigma_estimated : float
        The estimated value of the standard deviation parameter.
    References
    ----------
    Nelder, J A, and R Mead. 1965. A Simplex Method for Function Minimization. The Computer Journal 7: 308-13.
    
    Examples
    --------
    >>> data = [1.2, np.nan, 2.3, 3.4, np.nan]
    >>> mu_estimated,sigma_estimated = estimate_normal_params(data)
    >>> print(f"Estimated mu: {mu_estimated:.2f}")
    Estimated mu: 2.30
    >>> print(f"Estimated sigma: {sigma_estimated:.2f}")
    Estimated sigma: 0.88
   """

    # Create a copy of the input Series to avoid modifying it 
    y_series = y_series.copy()
    
    # Define initial guesses for the parameters
    mu_guess = np.mean(y_series)
    sigma_guess = np.std(y_series)

    # Minimize the negative log-likelihood
    result = minimize(lambda params: neg_log_likelihood(params=params, y_array=y_series.values),
                      x0=[mu_guess,sigma_guess],
                      method='Nelder-Mead')
    
    mu_estimated, sigma_estimated = result.x
    
    return mu_estimated, sigma_estimated

In [372]:
def interpolate_stock_prices(
        y_series: pd.Series,
        log_transform: bool =True
        )->pd.Series:
    """
    Interpolates missing values in a time series of stock prices using Maximum Likelihood Estimation (MLE).

    This function takes in a Pandas Series representing a time series of stock prices and interpolates any missing values using MLE. The function first 
    makes the series stationary by differencing it up to two times. Then it estimates the parameters of the normal distribution that best fits the 
    stationary series using MLE. The missing values are then filled in with the estimated mean of this distribution.

    If `log_transform` is set to `True`, the function will apply a log transformation to the input series before making it stationary and estimating its 
    parameters. This can help stabilize variance and make the data more normally distributed. After filling in the missing values, the function applies 
    a LOWESS smoother to re-estimate any duplicated MLE values. 
    
    Finally, it reverses any differencing and log transformations that were applied to return a series with interpolated values.

    Parameters
    ----------
    y_series : pd.Series
        A Pandas Series representing a time series of stock prices.
    log_transform : bool, optional, default = True
        Whether or not to apply a log transformation to the input series before making it stationary and estimating its parameters. 

    Returns
    -------
    pd.Series
        A Pandas Series with interpolated values for any missing data in `y_series`.
    
    Examples
    --------
    >>> y_series = pd.Series([100, np.nan, 102, 0, 0, 103, np.nan, 105, 0.0, 106, np.nan, 108, 109])
    >>> interpolated_series = interpolate_stock_prices(y_series, log_transform=True)
    >>> print(interpolated_series)
    """
    # Create a copy of the input Series to avoid modifying it 
    y_series = y_series.copy()
    y_nan = y_series.copy()

    # Replace 0 values with NaN
    y_nan[y_nan == 0] = np.nan
    
    # Remove NaNs and zeros
    no_nan_y = y_series[(y_series != 0) & (y_series.notnull())]
    
    # If Log transform series
    if log_transform:
        no_nan_y = np.log1p(no_nan_y)
        
    # Make the series stationary
    stationary_series, num_diff = make_stationary(y_series=no_nan_y, max_diff=2)
        
    # Use Maximum Likelihood Estimation (MLE) to estimate the parameters 
    mu_estimated, sigma_estimated = estimate_normal_params(y_series=stationary_series)
    
    # Replace values in y with stationary values 
    y = y_nan.copy()
    y.update(stationary_series)
    
    # Use the estimated parameters to fill in missing values with the estimated values for mu. 
    series_filled = [x if ~np.isnan(x) else mu_estimated for x in y]

    # Reverse differencing 
    for _ in range(num_diff):
        series_filled = np.cumsum(series_filled)

    # Inverse-transform Log transform 
    if log_transform:
        series_filled = np.expm1(series_filled)
        
    # Lowess smoothed series to re-estimate duplicated MLE values
    filled_smooth = smooth_lowess(y_series=pd.Series(series_filled), smoothing_window=4, smoothing_iterations=3)
    
    # Replace MLE values in NaN's place with smoothed values 
    return y_nan.fillna(filled_smooth)

In [373]:
y_series = pd.Series([100, np.nan, 102, 0, 0, 103, np.nan, 105, 0.0, 106, np.nan, 108, 109])
interpolated_series = interpolate_stock_prices(y_series, log_transform=True)

pd.DataFrame({'y':y_series,'y_hat':y_hat})

,y,y_hat
0,100.0,100.000000
1,NaN,101.596718
2,102.0,102.000000
3,0.0,103.926193
4,0.0,104.206711
5,103.0,103.000000
6,NaN,104.293114
7,105.0,105.000000
8,0.0,105.142421
9,106.0,106.000000


In [152]:
y_series = pd.Series([100, np.nan, 102, 0, 0, 103, np.nan, 105, 0.0, 106, np.nan, 108, 109])
stationary_series = pd.Series(
    data=[4.615121, 4.634729, 4.644391, 4.663439, 4.672829, 4.691348, 4.700480],
    index=[0,2,5,7,9,11,12]
)      
y_series.update(stationary_series)
y_series



0     4.615121
1          NaN
2     4.634729
3     0.000000
4     0.000000
5     4.644391
6          NaN
7     4.663439
8     0.000000
9     4.672829
10         NaN
11    4.691348
12    4.700480
dtype: float64

In [150]:
stationary_series = pd.Series(
    data=[4.615121, 4.634729, 4.644391, 4.663439, 4.672829, 4.691348, 4.700480],
    index=[0,2,5,7,9,11,12]
)      
stationary_series

0     4.615121
2     4.634729
5     4.644391
7     4.663439
9     4.672829
11    4.691348
12    4.700480
dtype: float64

In [133]:
test_series = pd.Series(np.arange(0,7,1))

In [136]:
[val for val in y_series]

[100.0, nan, 102.0, 0.0, 0.0, 103.0, nan, 105.0, 0.0, 106.0, nan, 108.0, 109.0]

In [ ]:

# Make an array and remove NaN values and Zeros 
#y_ = y_series['y']
#y_array = y_array[(~np.isnan(y_array))&(y_array != 0)]


In [73]:
y_series = pd.DataFrame({'y':[100, np.nan, 102, 0, 0, 103, np.nan, 105, 0.0, 106, np.nan, 108, 109]})['y']

# Create a copy of the input Series to avoid modifying it 
y_series = y_series.copy()

# Remove NaNs and zeros and keep the index
y_series = y_series[(y_series != 0) & (y_series.notnull())]

# Log transform series
y_series = np.log1p(y_series)

# Make the resuling array stationary
diff_series,i = make_stationary(y_series=y_series, max_diff=2)

# define initial guesses mu and sigma

mu_guess2 = np.mean(y_array)
sigma_guess2 = np.std(y_array)

print(diff_series)
print(f'i: {i}')



0     4.615121
2     4.634729
5     4.644391
7     4.663439
9     4.672829
11    4.691348
12    4.700480
Name: y, dtype: float64
i: 0


In [ ]:


stationary_series
# # define initial guesses for the parameters
# mu_guess2 = np.mean(y_array)
# sigma_guess2 = np.std(y_array)

In [58]:
print(mu_guess, sigma_guess)
print(mu_guess2, sigma_gue`ss2)

73.3 48.052159160645424
104.71428571428571 3.0101867865293537


In [ ]:

def interpolate_stock_prices(y_series):
    
    # Create a copy of the input Series to avoid modifying it 
    y_series = y_series.copy()
    
    # Convert to a numpy array
    y_array = y_series.to_numpy()

    # Remove NaN values adn Zeros
    y_array = y_array[(~np.isnan(y_array))|(y_array != 0)]
    #y_array = y_array[y_array != 0]

In [375]:
import multiprocessing

In [376]:
print(f"Number of CPU cores: {multiprocessing.cpu_count()}")

Number of CPU cores: 12


In [377]:
import tqdm

ModuleNotFoundError: No module named 'tqdm'